# VotecounterTest
A script for setting up and executing performance tests on votecounters.

## Dependencies

In [1]:
import json
import time
import numpy as np
from VoteCount import VoteCount
import VoteCounter
from tqdm import tqdm

# to help generate formatted output representations
from IPython.core.display import display, HTML
import markdown2 as md

def html(markdown_string):
    display(HTML(md.markdown(markdown_string)))

## Helper Function(s)

In [2]:
def _relevantGameInfo(game):
    link = game[:game.find('\n')]
    number = (link[link.find('&t=')+3:] if link.count('&')==1 
                      else link[link.find('&t=')+3:link.rfind('&')])
    slots, players, correct = [], [], None
    for line in game[game.find('\nPlayers\n')+9:].split('\n'):
        line = line.split(', ')
        players += line[0].split(' replaced ')
        slots.append(line[0].split(' replaced ')) 
        if (line[2].lower().count('lynched') > 0 and
            line[2].lower().count('day 1') > 0):
            correct = slots[-1]
    return slots, players, correct, number

## VotecounterTest

In [ ]:
#parameters
archive = '../data/archive.txt'
votecounter = VoteCounter
start_index = 0
end_index = 0

# open game archive, separate by game
with open(archive, encoding='utf-8') as f:
    games = f.read().split('\n\n\n')  
    
# process votes in each game's posts until a lynch found
# then store information about votecounter's performance
results, success, t0, total = {}, 0, time.time(), 0
end_index = end_index if end_index else len(games)  
for game_index, game in enumerate(games[start_index:end_index]):
    print()
    print('\n'.join(game.split('\n')[:2]))
    slots, players, correct, number = _relevantGameInfo(game)
    #print(slots, players, correct, number)
    votecount = VoteCount(slots, meta={'correct': correct})
    votecounter = VoteCounter.VoteExtracter(players=players)

    # collect gameposts associated with game number
    with open('../data/posts/{}.jsonl'.format(number)) as f:
        gameposts = [dict(t) for t in {tuple(d.items()) for d in [json.loads(l) for l in f]}]
        gameposts = sorted(gameposts, key=lambda x: (int(x['thread']), int(x['number'])))
        
    tgame = time.time()
    for post in gameposts:
        # done if voters have made a choice already
        if votecount.choice:
            break

        # ignore posts not made by players
        if players.count(post['user']) == 0:
            continue

        # update votecount for each vote found by votecounter
        for voted in votecounter.fromPost(post):
            votecount.update(post['user'], voted, post['number'])
            if votecount.choice:
                success += votecount.choice == correct
                break
    results[number] = votecount
    total += 1
    print(game_index + start_index, number, success, total, votecount.choice == correct, time.time()-tgame)
    
print(success/float(end_index-start_index), time.time()-t0)

## Analyze Results

In [ ]:
# parameters
game_index = 222
postnumber = 432
postnumber = str(postnumber)

# open game archive, separate by game
with open(archive, encoding='utf-8') as f:
    games = f.read().split('\n\n\n')  

# generate votecounter w/ specified players
slots, players, correct, number = _relevantGameInfo(games[game_index])
html('# {}'.format(number))
print(games[game_index])
print()
print('Choice:', results[number].choice)
print('Correct:', results[number].meta['correct'])
votecount = VoteCount(slots, meta={'correct': correct})
votecounter = VoteCounter.VoteExtracter(players=players)

# collect gameposts associated with game number
with open('../data/posts/{}.jsonl'.format(number)) as f:
    gameposts = [dict(t) for t in {tuple(d.items()) for d in [json.loads(l) for l in f]}]
    gameposts = sorted(gameposts, key=lambda x: (int(x['thread']), int(x['number'])))
    
# find and display selected post along with votecounter output for it
html('# Post {}'.format(postnumber))
post = next(item for item in gameposts if item["number"] == postnumber)
print('Extracted Votes:', list(votecounter.fromPost(post)))
print()
print(post)
print()
display(HTML(post['content']))
    
# display votecount up to indicated post number
html('# Current Votecount (Up to {})'.format(postnumber))
for post in gameposts[:int(postnumber)]:
    # done if voters have made a choice already
    if votecount.choice:
        break

    # ignore posts not made by players
    if players.count(post['user']) == 0:
        continue

    # update votecount for each vote found by votecounter
    for voted in votecounter.fromPost(post):
        votecount.update(post['user'], voted, post['number'])
        if votecount.choice:
            success += votecount.choice == correct
            break
            
current_votecount = votecount.todict()
for each in current_votecount:
    if current_votecount[each]:
        print(each, '-', len(current_votecount[each]))
        for voter in current_votecount[each]:
            print(voter)
        print()

# display final votecount
html('# Final Votecount')
final_votecount = results[number].todict()
for each in final_votecount:
    if final_votecount[each]:
        print(each, '-', len(final_votecount[each]))
        for voter in final_votecount[each]:
            print(voter)
        print()
        
# display final votelog
html('# Vote Log')
votelog = results[number].votelog.copy()
for index, each in enumerate(votelog):
    each = each.split()
    each[-1] = '[{}]({})'.format(each[-1], next(item for item in gameposts if item["number"] == each[-1])['pagelink'])
    votelog[index] = ' '.join(each)
html('  \n'.join(reversed(votelog)))

In [ ]:
gameposts[:int(postnumber)]